In [44]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [45]:
#i need that data that contaons just 3's and 7's
#that is my first step
#importing all i need to make my life easy


#hide
from fastai.vision.all import *
#from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [46]:
path = untar_data(URLs.MNIST_SAMPLE)


In [47]:
path.ls()

(#3) [Path('/root/.fastai/data/mnist_sample/valid'),Path('/root/.fastai/data/mnist_sample/labels.csv'),Path('/root/.fastai/data/mnist_sample/train')]

In [48]:
(path/'train').ls()


(#2) [Path('/root/.fastai/data/mnist_sample/train/7'),Path('/root/.fastai/data/mnist_sample/train/3')]

In [49]:
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()


In [50]:
seven_tensors = [tensor(Image.open(o)) for o in sevens]
three_tensors = [tensor(Image.open(o)) for o in threes]
len(three_tensors),len(seven_tensors)

(6131, 6265)

In [51]:
#I need to stack the images untop of one another, right now , all we have is a list and 
# not neccessarily useful.
stacked_threes = torch.stack(three_tensors).float()/255
stacked_sevens = torch.stack(seven_tensors).float()/255


In [52]:
stacked_threes.shape

torch.Size([6131, 28, 28])

In [53]:
#now i need to find a way to concatenate all of these.
train_x = torch.cat([stacked_threes, stacked_sevens])

In [54]:
train_x.shape

torch.Size([12396, 28, 28])

In [55]:
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)
train_x.shape

torch.Size([12396, 784])

In [56]:
train_y = tensor([1]*len(stacked_threes) + [0]*len(stacked_sevens))
train_y.shape

torch.Size([12396])

In [57]:
dset = list(zip(train_x, train_y))
x , y = dset[0]
x.shape, y

(torch.Size([784]), tensor(1))

In [58]:
df = DataLoader(dset, batch_size = 256)
xb, yb =first(df)
xb.shape, yb.shape

(torch.Size([256, 784]), torch.Size([256]))

In [59]:
valid_3_tens = torch.stack([tensor(Image.open(o))
                            for o in (path/'valid'/'3').ls()])
valid_3_tens = valid_3_tens.float()/255
valid_7_tens = torch.stack([tensor(Image.open(o))
                            for o in (path/'valid'/'7').ls()])
valid_7_tens = valid_7_tens.float()/255
valid_3_tens.shape,valid_7_tens.shape


(torch.Size([1010, 28, 28]), torch.Size([1028, 28, 28]))

In [60]:
valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
valid_y = tensor([1]*len(valid_3_tens) + [0]*len(valid_7_tens)).unsqueeze(1)
valid_dset = list(zip(valid_x,valid_y))

In [61]:
valid_dl = DataLoader(valid_dset, batch_size=256)

__STEP 1 : INITIALIZE WEIGHTS__

In [62]:
def init_params(size, std =1.0):
    return (torch.randn(size)*std).requires_grad_()
    
    

In [63]:
weights = init_params(28*28 ,1)
bias = init_params(1)
weights.shape, bias.shape

(torch.Size([784]), torch.Size([1]))

__STEP 2 : PREDICTION FUNCTION__

In [64]:
def linear1(xb):
    preds =  xb@weights + bias
    return preds
    

In [65]:
#i decide to just test my code as I go 
batch = train_x[:4]

In [66]:
preds = linear1(batch)
preds

tensor([-15.6094, -20.3102,   4.7220,  -6.5382], grad_fn=<AddBackward0>)

In [67]:
#i can decide to just do do this with normal equation too without matrix multiplication
(batch[0]*weights.T).sum() + bias


tensor([-15.6094], grad_fn=<AddBackward0>)

__STEP 3 : LOSS FUNCTION__

In [68]:
def mnist_loss(preds, trgts):
    preds = preds.sigmoid()
    return torch.where(trgts==1.0, 1-preds , preds).mean()
    

__STEP 4 : CALCULATE GRADIENTS FUNCTION__ 

In [69]:
 def calc_grad(xb, yb, model):
     preds = model(xb)
     loss = mnist_loss(preds, yb)
     loss.backward()

__STEP 5 : BATCH_ACCURACY FUNCTION__

In [70]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    corrects = (preds> 0.5).float() == yb
    return corrects.float().mean()
    

In [71]:
def validate_epoch(model):
    accs = [batch_accuracy(model(xb), yb) for xb, yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)

__STEP 6 : TRAIN AN EPOCH__

In [72]:
def train_epoch(params, lr, model):
    for xb , yb in df:
        calc_grad(xb , yb ,model)
        for p in params:
            p.data -= p.grad * lr 
            p.grad.zero_()

In [73]:
lr =1.0
params = weights, bias
train_epoch(params, lr, linear1)
validate_epoch(linear1)

0.5914

In [74]:

train_epoch(params, lr, linear1)
validate_epoch(linear1)

0.6926

__STEP 6: RUN FOR MULTIPLE EPOCHS__

In [75]:
for i in range(30):
    train_epoch(params , lr, linear1)
    print(validate_epoch(linear1), end= ' ')

0.8742 0.928 0.9424 0.948 0.9518 0.9537 0.9547 0.9566 0.9575 0.9589 0.9595 0.9605 0.9614 0.9619 0.9614 0.9614 0.9619 0.9619 0.9629 0.9639 0.9639 0.9643 0.9648 0.9653 0.9653 0.9643 0.9648 0.9643 0.9643 0.9648 

__STEP 7 : BASIC OPTIMIZATION FUNCTION__

In [76]:
#luckily pytorch has nn.Linear which is  a module that has a linear fucntiion with parameters and weights

In [92]:
model = nn.Linear(28*28, 1)
weights, bias =model.parameters()
weights.shape , bias.shape 

(torch.Size([1, 784]), torch.Size([1]))

In [93]:
class BasicOptim:
    def __init__(self, params , lr):
        self.params = list(params)
        self.lr = lr
    def step(self):
        for p in self.params:
            p.data -= p.grad*lr
    def zero_grad(self):
        for p in self.params:
            p.grad.zero_()

In [94]:
#create an object from from this class
opt = BasicOptim(model.parameters(), lr)


In [95]:
def train_epoch(model):
    for xb , yb in df:
        calc_grad(xb , yb ,model)
        opt.step()
        opt.zero_grad()

In [96]:
validate_epoch(model)

0.508

In [97]:
def train_model(model, epoch):
    for i in range(epoch):
        train_epoch(model)
        print(validate_epoch(model),end =' ' )
    

In [98]:
train_model(model, 20)

0.4932 0.8911 0.8178 0.9091 0.9331 0.9472 0.9545 0.9614 0.9662 0.9672 0.9692 0.9726 0.9731 0.975 0.9755 0.976 0.9775 0.978 0.9785 0.9785 

In [99]:
#actually fastai gives a class to do basic optim  which is  SGD!
model = nn.Linear(28*28,1)
opt = SGD(model.parameters(), lr)
train_model(model, 20)


0.4932 0.915 0.7988 0.9023 0.9296 0.9453 0.9555 0.9609 0.9658 0.9677 0.9692 0.9716 0.9731 0.975 0.976 0.976 0.9775 0.978 0.9785 0.9785 

In [100]:
dls = DataLoaders(df, valid_dl)

In [101]:
learn = Learner(dls, nn.Linear(28*28,1), opt_func=SGD, 
                loss_func = mnist_loss,metrics=batch_accuracy)


In [102]:
learn.fit(10, lr=lr)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.637289,0.502972,0.495584,00:00
1,0.401899,0.269129,0.750245,00:00
2,0.154148,0.160732,0.854269,00:00
3,0.071203,0.100186,0.917076,00:00
4,0.040575,0.074603,0.934740,00:00
5,0.028507,0.060455,0.948970,00:00
6,0.023433,0.051546,0.956330,00:00
7,0.021085,0.045546,0.963199,00:00
8,0.019835,0.041279,0.965653,00:00
9,0.019051,0.038108,0.967125,00:00
